In [ ]:
from flask import Flask, request, jsonify
import joblib
import numpy as np

# Load your pre-trained model
model = joblib.load('prediction_model.pkl')  # Replace with your model file path

# Initialize Flask app
app = Flask(__name__)

@app.route('/', methods=['GET'])
def home():
    """Welcome message for the API."""
    return jsonify({
        "message": "Welcome to the Machine Downtime Prediction API!",
        "endpoints": {
            "/predict": "POST - Predict machine downtime",
            "/health": "GET - API health check",
            "/info": "GET - Model or dataset information"
        }
    })

@app.route('/health', methods=['GET'])
def health():
    """Health check endpoint."""
    return jsonify({"status": "API is running", "model_loaded": True})

@app.route('/info', methods=['GET'])
def info():
    """Information about the model or dataset."""
    return jsonify({
        "model": "Random Forest Classifier",
        "version": "1.0",
        "description": "Predict machine downtime or production defects using manufacturing data.",
        "features": ["Air temperature [K]", "Process temperature [K]", "Rotational speed [rpm]", "Torque [Nm]", "Tool wear [min]"]
    })

@app.route('/predict', methods=['POST'])
def predict():
    """Prediction endpoint."""
    try:
        # Parse JSON input
        data = request.get_json()

        # Extract required features
        air_temperature = data.get('Air temperature [K]')
        process_temperature = data.get('Process temperature [K]')
        rotational_speed = data.get('Rotational speed [rpm]')
        torque = data.get('Torque [Nm]')
        tool_wear = data.get('Tool wear [min]')

        # Check for missing data
        if air_temperature is None or process_temperature is None or rotational_speed is None or torque is None or tool_wear is None:
            return jsonify({"error": "Data is missing"}), 400

        # Prepare input for the model
        input_features = np.array([[air_temperature, process_temperature, rotational_speed, torque, tool_wear]])

        # Predict using the loaded model
        prediction = model.predict(input_features)

        # Map prediction to class labels
        downtime = "Yes" if prediction[0] == 1 else "No"

        return jsonify({
            "air_temperature": air_temperature,
            "process_temperature": process_temperature,
            "rotational_speed": rotational_speed,
            "torque": torque,
            "tool_wear": tool_wear,
            "Downtime": downtime
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)
